In [1]:
from transformers import BertTokenizer, BertModel
import torch
import torchvision
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

/Users/weiceica/Desktop/URPS/LLM4GeoInfo/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
cities = pd.read_csv('cities.csv')

/var/folders/6n/tkyy6kcj11s3n3dzjcx0c09r0000gn/T/ipykernel_88814/3260777171.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
cities.head()

,Unnamed: 0,city_st,lat,lng,population,lat_prompt,lng_prompt,distance_km
0,0,"New York, NY",40.6943,-73.9249,18713220,40.7128,-74.0060,7.139212
1,1,"Los Angeles, CA",34.1139,-118.4068,12750807,34.0522,-118.2437,16.513283
2,2,"Chicago, IL",41.8373,-87.6862,8604203,41.8781,-87.6298,6.511529
3,3,"Miami, FL",25.7839,-80.2102,6445545,25.7617,-80.1918,3.080308
4,4,"Dallas, TX",32.7936,-96.7662,5743938,32.7767,-96.7970,3.438239


In [4]:
cities = cities[["city_st", "lat", "lng", "population"]]

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
cities['city_tokens'] = cities['city_st'].apply(lambda x: tokenizer.encode(x))
cities.head()

,city_st,lat,lng,population,city_tokens
0,"New York, NY",40.6943,-73.9249,18713220,"[101, 2047, 2259, 1010, 6396, 102]"
1,"Los Angeles, CA",34.1139,-118.4068,12750807,"[101, 3050, 3349, 1010, 6187, 102]"
2,"Chicago, IL",41.8373,-87.6862,8604203,"[101, 3190, 1010, 6335, 102]"
3,"Miami, FL",25.7839,-80.2102,6445545,"[101, 5631, 1010, 13109, 102]"
4,"Dallas, TX",32.7936,-96.7662,5743938,"[101, 5759, 1010, 19067, 102]"


In [6]:
import torch, torchvision
model = BertModel.from_pretrained('bert-base-uncased')
def get_embedding(tokens):
    with torch.no_grad():
        outputs = model(torch.tensor(tokens).unsqueeze(0))
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding
cities['city_embeddings'] = cities['city_tokens'].apply(get_embedding)

In [7]:
cities.head()

,city_st,lat,lng,population,city_tokens,city_embeddings
0,"New York, NY",40.6943,-73.9249,18713220,"[101, 2047, 2259, 1010, 6396, 102]","[-0.03927358, 0.030132243, -0.19327933, -0.076..."
1,"Los Angeles, CA",34.1139,-118.4068,12750807,"[101, 3050, 3349, 1010, 6187, 102]","[0.21018803, 0.07478464, -0.05120385, 0.056763..."
2,"Chicago, IL",41.8373,-87.6862,8604203,"[101, 3190, 1010, 6335, 102]","[0.00028579234, -0.07842561, -0.37666678, -0.0..."
3,"Miami, FL",25.7839,-80.2102,6445545,"[101, 5631, 1010, 13109, 102]","[-0.06461823, -0.10884482, -0.0012982726, 0.09..."
4,"Dallas, TX",32.7936,-96.7662,5743938,"[101, 5759, 1010, 19067, 102]","[0.15536723, -0.189955, -0.14865291, -0.166116..."


In [8]:
cities.to_csv("city_bert_embeddings.csv")